<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/GPTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 저축은행

## 1.1. 신용점수별 금리

- 조건
  - 가계신용대출, 저축은행별금리현황 -> 취급금액전체, 2025년 1월

In [ ]:
import requests
import json

def crawl_fsb_data(year, month):
  """
  fsb.or.kr 웹사이트에서 데이터를 크롤링하는 함수

  Args:
    year (int): 년도
    month (int): 월

  Returns:
    dict: 크롤링된 데이터 (JSON 형식)
  """

  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  if response.status_code == 200:
    try:
      # 응답 데이터를 JSON 형식으로 파싱
      data = response.json()
      return data
    except json.JSONDecodeError:
      print("Error: Invalid JSON response")
      return None
  else:
    print(f"Error: HTTP {response.status_code}")
    return None

if __name__ == "__main__":
  year = 2025
  month = 1

  crawled_data = crawl_fsb_data(year, month)

  if crawled_data:
    # 크롤링된 데이터 출력 (예시)
    print(crawled_data)

    # TODO: 크롤링된 데이터를 파일에 저장하거나 추가 처리
  else:
    print("Failed to crawl data.")

In [ ]:
def create_dataframe_from_json(json_data):
  """
  JSON 데이터를 입력받아 특정 열을 가진 Pandas DataFrame으로 변환합니다.

  Args:
    json_data: JSON 형식의 문자열 또는 Python 객체 (dict, list)

  Returns:
    Pandas DataFrame: 필요한 열을 가진 DataFrame. 오류 발생 시 None 반환.
  """
  try:
    # JSON 데이터 파싱
    if isinstance(json_data, str):
      data = json.loads(json_data)
    else:  # dict or list
      data = json_data

    # 데이터 추출 및 DataFrame 생성
    df = pd.DataFrame(data['REC'])
    df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
              'A_RATE3', 'A_RATE_AVE']]

    return df

  except (json.JSONDecodeError, KeyError, TypeError) as e:
    print(f"Error processing JSON data: {e}")
    return None

## 1.2. 금리대별 취급비중

In [ ]:
import requests
import json

def fetch_fsb_loan_data(year, month):
    """
    금융위원회 대출금리 공시 페이지에서 데이터를 받아오는 함수 (파싱 없이).

    Args:
        year (int): 년도 (YYYY 형식)
        month (int): 월 (MM 형식)

    Returns:
        str: JSON 응답 문자열
        None: 오류 발생 시
    """

    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    try:
        response = requests.post(url, data={"_JSON_": json.dumps(payload)})
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

        return response.text  # JSON 응답 문자열 반환

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# 예시 사용
year = 2025
month = 1

data = fetch_fsb_loan_data(year, month)

if data:
    print(data)  # JSON 문자열 출력 (raw data)
    # 필요하다면 이후에 JSON 파싱 및 데이터 추출 작업 수행
    try:
        json_data = json.loads(data) #JSON 파싱
        #print(json.dumps(json_data, indent=4, ensure_ascii=False)) #JSON 데이터 예쁘게 출력
    except json.JSONDecodeError:
        print("Error: Invalid JSON response")
else:
    print("Failed to fetch data.")

In [7]:
import json
import pandas as pd

def process_json_data(json_string):
    """
    JSON 문자열을 파싱하여 DataFrame으로 변환하고, 원하는 열만 남기는 함수.

    Args:
        json_string (str): JSON 문자열

    Returns:
        pandas.DataFrame: 변환된 DataFrame (원하는 열만 남은 상태)
        None: JSON 파싱 오류 또는 데이터 추출 오류 시
    """
    try:
        data = json.loads(json_string)

        if not isinstance(data, dict) or "REC" not in data or not isinstance(data["REC"], list):
            print("Invalid JSON format: 'REC' key not found or not a list.")
            return None

        rec_data = data["REC"]
        df = pd.DataFrame(rec_data)  # 모든 열을 포함하는 DataFrame 생성

        # 원하는 열만 선택
        desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                            ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
        df = df[desired_columns] #원하는 열만 남기기

        return df

    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None
    except KeyError as e:  # KeyError 추가
        print(f"Error: Key '{e}' not found in JSON data")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

process_json_data(data)

,BANK_NAME,SUBMIT_MONTH,HANDING_WEIGHT_10,HANDING_WEIGHT_12,HANDING_WEIGHT_14,HANDING_WEIGHT_16,HANDING_WEIGHT_18,HANDING_WEIGHT_20
0,BNK,202501,0.69,27.67,36.64,34.99,0.01,0
1,DB,202501,NaN,67.98,28,3.03,0.99,NaN
2,IBK,202501,NaN,11.87,36.15,12.77,11.74,27.47
3,JT,202501,0.18,29.79,13.19,46.16,7.71,2.97
4,JT친애,202501,NaN,0.59,36.54,37.86,15.63,9.38
5,KB,202501,1.73,6.47,24.96,17.6,18.79,30.45
6,NH,202501,0,49.87,28.55,15.68,5.27,0.63
7,OK,202501,0.01,0.32,5.18,29.15,31.99,33.35
8,OSB,202501,NaN,NaN,29.8,30.55,7.97,31.68
9,SBI,202501,0.96,11.64,35.99,28.17,16.19,7.05


# 2. 신용카드

## 2.1. 신용점수별 금리

In [27]:
import requests
import certifi
import ssl


class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)


def fetch_crefia_data(cgcYyyy):
    """
    여신금융협회 공시 페이지에서 데이터를 받아오는 함수 (파싱 없이).

    Args:
        cgcSeq (int): cgcSeq 값
        cgcMode (int): cgcMode 값
        cgcYyyy (int): cgcYyyy 값
        mcSeq_list (list): mcSeq 값들의 리스트

    Returns:
        str: 응답 문자열
        None: 오류 발생 시
    """

    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": cgcYyyy,
        "mcSeq": []
      }

      # mcSeq 파라미터 처리
      # for mcSeq in mcSeq_list:
      #     params[f"mcSeq"] = mcSeq  # mcSeq는 여러 번 반복될 수 있음

      try:
          response = s.get(url, params=params, verify=certifi.where())
          response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

      except requests.exceptions.RequestException as e:
          print(f"Error: {e}")
      except Exception as e:
          print(f"An unexpected error occurred: {e}")

    response


# 예시 사용

data = fetch_crefia_data(2024)

if data:
    print(data)  # 응답 문자열 출력 (raw data)
    # 필요하다면 이후에 파싱 및 데이터 추출 작업 수행
else:
    print("Failed to fetch data.")

{"mcSeq":[],"configListMm":[{"cgcYearMonth":"2024년 12월","cgcYyyy":"2024년","cgcSeq":1451,"cgcquarter":12,"cgcMm":"12월","cgcYearQuarter":"2024년 12분기","cgcYear":"2024","cgcPubDate":"2024.12.20"},{"cgcYearMonth":"2024년 11월","cgcYyyy":"2024년","cgcSeq":1439,"cgcquarter":11,"cgcMm":"11월","cgcYearQuarter":"2024년 11분기","cgcYear":"2024","cgcPubDate":"2024.11.20"},{"cgcYearMonth":"2024년 10월","cgcYyyy":"2024년","cgcSeq":1422,"cgcquarter":10,"cgcMm":"10월","cgcYearQuarter":"2024년 10분기","cgcYear":"2024","cgcPubDate":"2024.10.18"},{"cgcYearMonth":"2024년 9월","cgcYyyy":"2024년","cgcSeq":1411,"cgcquarter":9,"cgcMm":"09월","cgcYearQuarter":"2024년 9분기","cgcYear":"2024","cgcPubDate":"2024.09.20"},{"cgcYearMonth":"2024년 8월","cgcYyyy":"2024년","cgcSeq":1400,"cgcquarter":8,"cgcMm":"08월","cgcYearQuarter":"2024년 8분기","cgcYear":"2024","cgcPubDate":"2024.08.20"},{"cgcYearMonth":"2024년 7월","cgcYyyy":"2024년","cgcSeq":1383,"cgcquarter":7,"cgcMm":"07월","cgcYearQuarter":"2024년 7분기","cgcYear":"2024","cgcPubDate":"2024.07.19

In [42]:
# data = json.loads(data)
data = data['configListMm']

for item in data:
    if item['cgcquarter'] == 1:
        cgc_seq = item['cgcSeq']
        print(f"cgcquarter가 1일 때 cgcSeq: {cgc_seq}")
        break  # 첫 번째로 찾은 값만 출력하고 반복문 종료
else:
    print("cgcquarter가 1인 데이터를 찾을 수 없습니다.")

cgcquarter가 1일 때 cgcSeq: 1305


## 2.2. 금리대별 취급비중

SSL 오류: [SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1006)
